In [472]:
import pandas as pd
import numpy as np
import mojimoji
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer

In [473]:
train = pd.read_csv('../data/total_train.csv',encoding='utf-8-sig')
test = pd.read_csv('../data/total_test.csv',encoding='utf-8-sig')

In [474]:
train[train['y'] == 0]

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
1567,15699,0,2014,Ｊ１,第４節第１日,03/23(日),15:04,浦和レッズ,清水エスパルス,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／テレ玉


In [475]:
train.shape , test.shape

((1952, 11), (351, 10))

In [476]:
# absorp = pd.read_csv('../data/absorp4.csv')
# train = pd.merge(left=train,right=absorp,on=['stadium']).sort_values('id').reset_index(drop=True)
# test = pd.merge(left=test,right=absorp,on=['stadium']).sort_values('id').reset_index(drop=True)

In [477]:
train

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌
...,...,...,...,...,...,...,...,...,...,...,...
1947,16234,7842,2014,Ｊ２,第２４節第１日,07/30(水),19:04,ジュビロ磐田,カマタマーレ讃岐,ヤマハスタジアム（磐田）,スカパー！／スカパー！プレミアムサービス
1948,16235,5212,2014,Ｊ２,第２４節第１日,07/30(水),19:04,ＦＣ岐阜,ファジアーノ岡山,岐阜メモリアルセンター長良川競技場,スカパー！／スカパー！プレミアムサービス
1949,16236,3883,2014,Ｊ２,第２４節第１日,07/30(水),19:03,アビスパ福岡,湘南ベルマーレ,レベルファイブスタジアム,スカパー！／スカパー！プレミアムサービス
1950,16237,5028,2014,Ｊ２,第２４節第１日,07/30(水),19:03,大分トリニータ,栃木ＳＣ,大分銀行ドーム,スカパー！／スカパー！プレミアムサービス


In [478]:
train['week_day'] = train['gameday'].str[-2]
train['gameday'] = train['gameday'].str[:5]

test['week_day'] = test['gameday'].str[-2]
test['gameday'] = test['gameday'].str[:5]

In [479]:
## 祝日と休日だけ一つにしてしまいその他は一度放置
week_day = {'月':0,'火':1,'水':2,'木':3,'金':4,'土':5,'日':6,'休':7,'祝':7}

train['week_day'] = train['week_day'].map(week_day)
train['month'] = train['gameday'].str[:2].astype(int)
train['day'] = train['gameday'].str[-2:].astype(int)

test['week_day'] = test['week_day'].map(week_day)
test['month'] = test['gameday'].str[:2].astype(int)
test['day'] = test['gameday'].str[-2:].astype(int)

In [480]:
train['week_day'].value_counts()

6    807
5    785
7    192
2    127
4     30
3      5
0      4
1      2
Name: week_day, dtype: int64

In [481]:
## match節・stageの調整
train['match'] = train['match'].apply(lambda x: x[1:].rsplit('節')[0])
train['match'] = train['match'].apply(mojimoji.zen_to_han)

test['match'] = test['match'].apply(lambda x: x[1:].rsplit('節')[0])
test['match'] = test['match'].apply(mojimoji.zen_to_han)

In [482]:
train['stage'] = train['stage'].apply(mojimoji.zen_to_han).str[1]
test['stage'] = test['stage'].apply(mojimoji.zen_to_han).str[1]

train['tv'] = train['tv'].apply(mojimoji.zen_to_han).str.split('/')
test['tv'] = test['tv'].apply(mojimoji.zen_to_han).str.split('/')


In [483]:
##tvの各要素を見ていき、録画は削除、NHK,地方局はまとめる
#インデックスの都合上、先に録画を削除
for i in range(len(train['tv'])):
   train.loc[i,'tv_cnt'] = len(train.loc[i,'tv'])
   for j in range(len(train.loc[i,'tv'])):
      if '(録)' in train.loc[i,'tv'][j]:
        train.loc[i,'tv_cnt'] -=1
        train.loc[i,'tv'][j] = 'r'

for i in train['tv']:
   for j in range(len(i)):
       if 'NHK' in i[j]:
          i[j] = 'NHK'
       elif 'ｽｶﾊﾟｰ' in i[j] or 'e2' in i[j]:
          i[j] = 'ｽｶﾊﾟｰ'
       elif 'ﾃﾚﾋﾞ' in i[j] or '放送' in i[j] or 'KBS京都' in i[j] or 'TOKYO MX' in i[j] or 'ﾃﾚ玉' in i[j]:
          i[j] = '地方局'

for i in range(len(test['tv'])):
   test.loc[i,'tv_cnt'] = len(test.loc[i,'tv'])
   for j in range(len(test.loc[i,'tv'])):
      if '(録)' in test.loc[i,'tv'][j]:
        test.loc[i,'tv_cnt'] -=1
        test.loc[i,'tv'][j] = 'r'

for i in test['tv']:
   for j in range(len(i)):
       if 'NHK' in i[j]:
          i[j] = 'NHK'
       elif 'ｽｶﾊﾟｰ' in i[j] or 'e2' in i[j]:
          i[j] = 'ｽｶﾊﾟｰ'
       elif 'ﾃﾚﾋﾞ' in i[j] or '放送' in i[j] or 'KBS京都' in i[j] or 'TOKYO MX' in i[j] or 'ﾃﾚ玉' in i[j]:
          i[j] = '地方局'
      

In [484]:
mlb = MultiLabelBinarizer(sparse_output=True)
train = train.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(train.pop('tv')),
                index=train.index,
                columns=mlb.classes_))

train.drop(['r' , 'gameday','time'],axis =1,inplace = True)

test = test.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(test.pop('tv')),
                index=test.index,
                columns=mlb.classes_))

test.drop(['r' , 'gameday','time'],axis =1,inplace = True)

In [485]:
# チーム名などの変更を統一したいがデータの都合上先に連結させる
condition = pd.read_csv('../data/total_condition.csv')
stadium = pd.read_csv('../data/stadium_new.csv')

In [486]:
stadium.columns


Index(['name', 'address', 'capa', 'lat', 'lng', 'station', 'age', 'child',
       'population', 'area'],
      dtype='object')

In [487]:
train = pd.merge(left=train,right=condition,on='id')
train.drop(['home_team','away_team'],axis=1,inplace=True)

test = pd.merge(left=test,right=condition,on='id')
test.drop(['home_team','away_team'],axis=1,inplace=True)

In [488]:
test.columns

Index(['id', 'year', 'stage', 'match', 'home', 'away', 'stadium', 'week_day',
       'month', 'day', 'tv_cnt', 'BS-TBS', 'NHK', '地方局', 'ｽｶﾊﾟｰ', 'home_score',
       'away_score', 'weather', 'temperature', 'humidity', 'referee',
       'home_01', 'home_02', 'home_03', 'home_04', 'home_05', 'home_06',
       'home_07', 'home_08', 'home_09', 'home_10', 'home_11', 'away_01',
       'away_02', 'away_03', 'away_04', 'away_05', 'away_06', 'away_07',
       'away_08', 'away_09', 'away_10', 'away_11'],
      dtype='object')

In [489]:
absorp = pd.read_csv('../data/absorp.csv')[['year','home','stadium','absorp']]
test = pd.merge(left=test,right=absorp,on=['year','home','stadium']).sort_values('id')
train = pd.merge(left=train,right=absorp,on=['year','home','stadium']).sort_values('id')

# absorp = pd.read_csv('../data/absorp3.csv')
# test = pd.merge(left=test,right=absorp,on=['stadium']).sort_values('id')
# train = pd.merge(left=train,right=absorp,on=['stadium']).sort_values('id')

In [490]:
for i in range(len(train['home'])):
    if 'ザスパ草津' in train.loc[i,'home']:
        train.loc[i,'home'] = 'ザスパクサツ群馬'

for i in range(len(train['away'])):
    if 'ザスパ草津' in train.loc[i,'away']:
        train.loc[i,'away'] = 'ザスパクサツ群馬'

for i in range(len(test['home'])):
    if 'ザスパ草津' in test.loc[i,'home']:
        test.loc[i,'home'] = 'ザスパクサツ群馬'

for i in range(len(test['away'])):
    if 'ザスパ草津' in test.loc[i,'away']:
        test.loc[i,'away'] = 'ザスパクサツ群馬'

In [491]:
for condition in ["晴", "曇", "雨", "屋内", "雷", "雪", "霧", "のち", "時々", "一時"]:
    train["weather_{}".format(condition)] = train["weather"].str.contains(condition).apply(int)
    test["weather_{}".format(condition)] = test["weather"].str.contains(condition).apply(int)

In [492]:
## スタジアムとの連結
train = pd.merge(left=train,right=stadium,left_on='stadium',right_on ='name',how='inner')
train = train.sort_values(['id'])
train.drop(['name','weather'],axis=1,inplace=True)

test = pd.merge(left=test,right=stadium,left_on='stadium',right_on ='name',how='inner')
test = test.sort_values(['id'])
test.drop(['name','weather'],axis=1,inplace=True)

In [494]:
home_stadiums = {'ＦＣ岐阜' : '岐阜メモリアルセンター長良川競技場',
'ＦＣ町田ゼルビア' : '町田市立陸上競技場',
'ＦＣ東京' : '味の素スタジアム',
'Ｖ・ファーレン長崎' : '長崎県立総合運動公園陸上競技場',
'アビスパ福岡' : 'レベルファイブスタジアム',
'アルビレックス新潟' : 'デンカビッグスワンスタジアム',
'ヴァンフォーレ甲府' : '山梨中銀スタジアム',
'ヴィッセル神戸' : 'ノエビアスタジアム神戸',
'ガイナーレ鳥取' : 'とりぎんバードスタジアム',
'カターレ富山' : '富山県総合運動公園陸上競技場',
'カマタマーレ讃岐' : '香川県立丸亀競技場',
'ガンバ大阪' : '万博記念競技場',
'ギラヴァンツ北九州' : '北九州市立本城陸上競技場',
'コンサドーレ札幌' : '札幌ドーム',
'サガン鳥栖' : 'ベストアメニティスタジアム',
'ザスパクサツ群馬' : '正田醤油スタジアム群馬',
'サンフレッチェ広島' : 'エディオンスタジアム広島',
'ジェフユナイテッド千葉' : 'フクダ電子アリーナ',
'ジュビロ磐田' : 'ヤマハスタジアム（磐田）',
'セレッソ大阪' : 'キンチョウスタジアム',
'ファジアーノ岡山' : 'シティライトスタジアム',
'ベガルタ仙台' : 'ユアテックスタジアム仙台',
'モンテディオ山形' : 'ＮＤソフトスタジアム山形',
'ロアッソ熊本' : 'うまかな・よかなスタジアム',
'愛媛ＦＣ' : 'ニンジニアスタジアム',
'浦和レッズ' : '埼玉スタジアム２００２',
'横浜Ｆ・マリノス' : '日産スタジアム',
'横浜ＦＣ' : 'ニッパツ三ツ沢球技場',
'京都サンガF.C.' : '京都市西京極総合運動公園陸上競技場兼球技場',
'鹿島アントラーズ' : '県立カシマサッカースタジアム',
'松本山雅ＦＣ' : '松本平広域公園総合球技場',
'湘南ベルマーレ' : 'Ｓｈｏｎａｎ　ＢＭＷスタジアム平塚',
'水戸ホーリーホック' : 'ケーズデンキスタジアム水戸',
'清水エスパルス' : 'ＩＡＩスタジアム日本平',
'川崎フロンターレ' : '等々力陸上競技場',
'大宮アルディージャ' : 'ＮＡＣＫ５スタジアム大宮',
'大分トリニータ' : '大分銀行ドーム',
'東京ヴェルディ' : '味の素スタジアム',
'徳島ヴォルティス' : '鳴門・大塚スポーツパーク　ポカリスエットスタジアム',
'栃木ＳＣ' : '栃木県グリーンスタジアム',
'柏レイソル' : '日立柏サッカー場',
'名古屋グランパス' : '豊田スタジアム'
}

In [495]:
test.columns

Index(['id', 'year', 'stage', 'match', 'home', 'away', 'stadium', 'week_day',
       'month', 'day', 'tv_cnt', 'BS-TBS', 'NHK', '地方局', 'ｽｶﾊﾟｰ', 'home_score',
       'away_score', 'temperature', 'humidity', 'referee', 'home_01',
       'home_02', 'home_03', 'home_04', 'home_05', 'home_06', 'home_07',
       'home_08', 'home_09', 'home_10', 'home_11', 'away_01', 'away_02',
       'away_03', 'away_04', 'away_05', 'away_06', 'away_07', 'away_08',
       'away_09', 'away_10', 'away_11', 'absorp', 'weather_晴', 'weather_曇',
       'weather_雨', 'weather_屋内', 'weather_雷', 'weather_雪', 'weather_霧',
       'weather_のち', 'weather_時々', 'weather_一時', 'address', 'capa', 'lat',
       'lng', 'station', 'age', 'child', 'population', 'area'],
      dtype='object')

In [496]:
train['home_stadium'] = train['home'].map(home_stadiums)
train = pd.merge(left=train,right=stadium[['name','lat','lng']],left_on='home_stadium',right_on ='name',how='inner')
train.rename(columns={'lat_x' : 'h_lat', 'lat_y' : 'lat', 'lng_x' : 'h_lng', 'lng_y' : 'lng'} ,inplace = True)

train['away_stadium'] = train['away'].map(home_stadiums)
train = pd.merge(left=train,right=stadium[['name','lat','lng']],left_on='away_stadium',right_on ='name',how='inner')

train.rename(columns={'lat_x' : 'lat', 'lat_y' : 'a_lat', 'lng_x' : 'lng', 'lng_y' : 'a_lng'} ,inplace = True)
train.drop(['name_x','name_y'],axis=1,inplace=True)

train.sort_values('id',inplace=True)
train.reset_index(drop=True,inplace=True)

test['home_stadium'] = test['home'].map(home_stadiums)
test = pd.merge(left=test,right=stadium[['name','lat','lng']],left_on='home_stadium',right_on ='name',how='inner')
test.rename(columns={'lat_x' : 'h_lat', 'lat_y' : 'lat', 'lng_x' : 'h_lng', 'lng_y' : 'lng'} ,inplace = True)


test['away_stadium'] = test['away'].map(home_stadiums)
test = pd.merge(left=test,right=stadium[['name','lat','lng']],left_on='away_stadium',right_on ='name',how='inner')

test.rename(columns={'lat_x' : 'lat', 'lat_y' : 'a_lat', 'lng_x' : 'lng', 'lng_y' : 'a_lng'} ,inplace = True)
test.drop(['name_x','name_y'],axis=1,inplace=True)

test.sort_values('id',inplace=True)
test.reset_index(drop=True,inplace=True)

In [497]:
train['humidity'] = train['humidity'].str[:-1].astype(float)
test['humidity'] = test['humidity'].str[:-1].astype(float)

def discomfort_index(T, H):
    return 0.81*T+0.01*H*(0.99*T - 14.3) + 46.3
    
train["discomfort"] = discomfort_index(train["temperature"], train["humidity"])
test["discomfort"] = discomfort_index(test["temperature"], test["humidity"])

In [498]:
train.shape

(1952, 70)

In [499]:
## チームの収益データの挿入

In [500]:
train['home'].unique()

array(['ベガルタ仙台', '名古屋グランパス', 'ガンバ大阪', 'サンフレッチェ広島', 'コンサドーレ札幌', 'サガン鳥栖',
       '川崎フロンターレ', '大宮アルディージャ', '柏レイソル', '鹿島アントラーズ', '浦和レッズ', '横浜Ｆ・マリノス',
       'アルビレックス新潟', 'セレッソ大阪', '清水エスパルス', 'ジュビロ磐田', 'ヴィッセル神戸', 'ＦＣ東京',
       '水戸ホーリーホック', '東京ヴェルディ', 'ヴァンフォーレ甲府', 'ＦＣ岐阜', 'ファジアーノ岡山', 'アビスパ福岡',
       '大分トリニータ', 'ジェフユナイテッド千葉', '愛媛ＦＣ', '湘南ベルマーレ', 'ギラヴァンツ北九州', '栃木ＳＣ',
       'ＦＣ町田ゼルビア', '松本山雅ＦＣ', '徳島ヴォルティス', 'ザスパクサツ群馬', '横浜ＦＣ', 'ロアッソ熊本',
       '京都サンガF.C.', 'モンテディオ山形', 'ガイナーレ鳥取', 'カターレ富山', 'Ｖ・ファーレン長崎',
       'カマタマーレ讃岐'], dtype=object)

## Teamの順位データの挿入

In [501]:
ranking = pd.read_csv('../data/ranking.csv')

In [502]:
ranking

,year,stage,match,team,ranking
0,2012,1,1,ヴィッセル神戸,1
1,2012,1,1,ＦＣ東京,2
2,2012,1,1,名古屋グランパス,2
3,2012,1,1,サンフレッチェ広島,2
4,2012,1,1,ベガルタ仙台,2
...,...,...,...,...,...
4603,2014,2,42,ＦＣ岐阜,18
4604,2014,2,42,愛媛ＦＣ,19
4605,2014,2,42,東京ヴェルディ,20
4606,2014,2,42,カマタマーレ讃岐,21


In [503]:
## さすがに1952 x 4608のデータをfor文で入れるのはやりすぎなので、
## mergeできるようにデータを操作する

##チームごとにソートしてひとつ前のデータを参照するようにする
# 一度実行すると欠損値かもしくは、最終順位が入っているはずなので、matchが1のものだけをすべて初期値とするリーグ所属チーム数に書き換える

ranking = ranking.sort_values(['team','year','match'])
ranking['ranking'] = ranking['ranking'].shift(periods= 1)

In [504]:
a = ranking[['year','stage','team']]
a.drop_duplicates().groupby(['year','stage']).count()

team
year stage      
2012 1        18
     2        22
2013 1        18
     2        22
2014 1        18
     2        22

In [505]:
_list =  ranking[ranking['match'] == 1].index

## matchが１のインデックスを取得し、回数を絞ってからloopにぶっこむ

for i in _list:
    ranking.loc[i,'ranking'] = 18 if ranking.loc[i,'stage'] == 1 else 22

ranking[ranking['match']==1]
ranking.sort_index(inplace=True)

In [506]:
for i in range(len(ranking['team'])):
    if 'ザスパ草津' in ranking.loc[i,'team']:
        ranking.loc[i,'team'] = 'ザスパクサツ群馬'

In [507]:
##調整ができたのでtrain,testデータに挿入する
train[['year','stage','match']] = train[['year','stage','match']].astype(int)
ranking[['year','stage','match']] = ranking[['year','stage','match']].astype(int)
test[['year','stage','match']] = test[['year','stage','match']].astype(int)

In [508]:
train = pd.merge(left= train,right= ranking,left_on=['year','stage','match','home'],right_on=['year','stage','match','team'])
test = pd.merge(left= test,right= ranking,left_on=['year','stage','match','home'],right_on=['year','stage','match','team'])
train.drop(['team'],axis=1,inplace=True)
test.drop(['team'],axis=1,inplace=True)

In [509]:

train = pd.merge(left= train,right= ranking,left_on=['year','stage','match','away'],right_on=['year','stage','match','team'])
test = pd.merge(left= test,right= ranking,left_on=['year','stage','match','away'],right_on=['year','stage','match','team'])
train.drop(['team'],axis=1,inplace=True)
test.drop(['team'],axis=1,inplace=True)

In [510]:
print

<function print>

## 選手の年俸データの挿入

In [511]:
player = pd.read_csv('../data/player.csv')
player.drop('Unnamed: 0',axis=1,inplace=True)
player.head(3)

,ranking,player_name,player_age,player_position,team_name,annual_salary
0,1,フォルラン,38,FW,セレッソ大阪,346500000
1,2,マルキーニョス,41,FW,ヴィッセル神戸,150000000
2,2,遠藤 保仁,37,MF,ガンバ大阪,150000000


In [512]:
for i in range(1,12):
    column = "home_%02d" % i
    for row,name in enumerate(train[column]):
            salary = player[player.player_name == name].annual_salary.values
            train.loc[row,column] = 2500000 if salary.size == 0 else salary
    column = "away_%02d" % i
    for row,name in enumerate(train[column]):
            salary = player[player.player_name == name].annual_salary.values
            train.loc[row,column] = 2500000 if salary.size == 0 else salary

train["home_all"] = train[["home_01","home_02","home_03","home_04","home_05","home_06","home_07","home_08","home_09","home_10","home_11"]].sum(axis=1) 
train["away_all"] = train[["away_01","away_02","away_03","away_04","away_05","away_06","away_07","away_08","away_09","away_10","away_11"]].sum(axis=1) 

for i in range(1,12):
    column = "home_%02d" % i
    for row,name in enumerate(test[column]):
            salary = player[player.player_name == name].annual_salary.values
            test.loc[row,column] = 2500000 if salary.size == 0 else salary
    column = "away_%02d" % i
    for row,name in enumerate(test[column]):
            salary = player[player.player_name == name].annual_salary.values
            test.loc[row,column] = 2500000 if salary.size == 0 else salary

test["home_all"] = test[["home_01","home_02","home_03","home_04","home_05","home_06","home_07","home_08","home_09","home_10","home_11"]].sum(axis=1) 
test["away_all"] = test[["away_01","away_02","away_03","away_04","away_05","away_06","away_07","away_08","away_09","away_10","away_11"]].sum(axis=1) 

In [513]:
stadium_label = train['stadium']
team_label = train['home']

for column in ['home','stadium','referee', 'address']:
        le = preprocessing.LabelEncoder()
        le.fit(train[column])
        train[column] = le.transform(train[column])


for column in ['referee', 'address']:
        le = preprocessing.LabelEncoder()
        le.fit(test[column])
        test[column] = le.transform(test[column])

In [514]:
stadium_transform = train['stadium']
stadium_map = pd.Series(stadium_transform.values,index=stadium_label)
stadium_map.drop_duplicates(keep="first",inplace=True)
stadium_map = stadium_map.to_dict()

team_transform = train['home']
team_map = pd.Series(team_transform.values,index=team_label)
team_map.drop_duplicates(keep="first",inplace=True)
team_map = team_map.to_dict()

In [515]:
train['away'] = train['away'].map(team_map)
train['home_stadium'] = train['home_stadium'].map(stadium_map)
train['away_stadium'] = train['away_stadium'].map(stadium_map)

test['home'] = test['home'].map(team_map)
test['away'] = test['away'].map(team_map)
test['stadium'] = test['stadium'].map(stadium_map)
test['home_stadium'] = test['home_stadium'].map(stadium_map)
test['away_stadium'] = test['away_stadium'].map(stadium_map)

In [516]:
def hubeny(lng1, lat1, lng2, lat2):
    
        Rx = 6378137.000
        Ry = 6356752.314140

        Dx = (lat1 - lat2)/360*2*np.pi
        Dy = (lng1 - lng2)/360*2*np.pi

        P = (lng1 + lng2)/360*np.pi

        E = np.sqrt((np.power(Rx, 2) - np.power(Ry, 2))/np.power(Rx, 2))
        W = np.sqrt(1-np.power(E, 2)*np.power(np.sin(P), 2))
        M = (Rx*(1-np.power(E, 2)))/(np.power(W, 3))
        N = Rx/W

        D = np.sqrt(np.power(Dy*M, 2) + np.power(Dx*N*np.cos(P), 2))
 
        return D

In [517]:
train['distance'] = hubeny(train['h_lng'],train['h_lat'],train['a_lng'],train['a_lat'])
test['distance'] = hubeny(test['h_lng'],test['h_lat'],test['a_lng'],test['a_lat'])

In [518]:
train.shape

(1952, 75)

In [519]:
j1 = train[train['stage'] == 1]
j2 = train[train['stage'] == 2]

week_absorp1 = j1[['week_day','absorp']].groupby('week_day').mean().to_dict()['absorp']
week_absorp2 = j2[['week_day','absorp']].groupby('week_day').mean().to_dict()['absorp']
week_absorp1[0] = week_absorp2[0]

In [520]:
for i in range(len(train)):
    if train.loc[i,'stage'] == 1:
        train.loc[i,'week_absorp'] = week_absorp1[train.loc[i,'week_day']]
    else : 
        train.loc[i,'week_absorp'] = week_absorp2[train.loc[i,'week_day']]
   

for i in range(len(test)):
    if test.loc[i,'stage'] == 1:
        test.loc[i,'week_absorp'] = week_absorp1[test.loc[i,'week_day']]
    else : 
        test.loc[i,'week_absorp'] = week_absorp2[test.loc[i,'week_day']]
   

In [521]:
train_add = pd.read_csv('../data/train_add.csv')
test_add = pd.read_csv('../data/test_add.csv')

In [522]:
train = pd.concat([train,train_add],axis=1)
test = pd.concat([test,test_add],axis=1)

In [523]:
train.shape

(1952, 79)

In [524]:
train.to_csv('../data/train_new2.csv',index=False)
test.to_csv('../data/test_new2.csv',index=False)